<div style="display: flex; align-items: flex-start; gap: 40px;">
  <div style="display: flex; align-items: center; gap: 10px;">
    <img src="https://www.linea.org.br/brand/linea-logo-color.svg" width="100" style="display: block;">
    <img src="https://cdn2.webdamdb.com/1280_c3PXjCZbPM23.png" width="180" style="display: block;">
  </div>
  
  <div style="margin-left: 20px;">
    <h2 style="margin: 0 0 10px 0; padding: 0;">Data Preparation on Object Tables<br> for PZ Compute pipeline</h2>
    Notebook for LIneA Open OnDemand Platform<br>
    Data Release: <a href="https://dp0-2.lsst.io/">Data Preview 0.2</a> <br>
    Authors: <a href="mailto:luigi.lcsilva@gmail.com">Luigi Lucas de Carvalho Silva</a>, <a href="mailto:julia@linea.org.br">Julia Gschwend</a> <br>
    Last verified to run: 2025-08-16 <br>
    Repository: <a href="https://github.com/linea-it/pz-lsst-inkind">linea-it/pz-lsst-inkind</a> <br>
  </div>
</div>

**tl;dr:**

This notebook prepares the photometric input catalog for the PZ Compute pipeline using LSST Object Catalog data. The process includes:
- Convert fluxes to magnitudes
- Apply dereddening corrections
- Round numerical values to eliminate excessive precision
- Mask invalid entries
- Apply quality cuts
- Save balanced parquet files with ~N rows (N chosen by user)  

## Imports

In [ ]:
################################### GENERAL ##########################################
import os
import gc
import glob
import time
import math
import yaml
import shutil
import logging
import tables_io
import numpy as np
import pandas as pd
import getpass
from datetime import datetime

##################################### DASK ###########################################
import dask
from dask import config as dask_config_class
from dask import dataframe as dd
from dask import delayed, compute
from dask.distributed import Client, performance_report, wait
from dask_jobqueue import SLURMCluster

##################################### ASTROPY ###########################################
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

In [ ]:
with open("config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

globals().update(cfg)

# Setups

Making the setup of dustmaps.

In [ ]:
##################################### DUSTMAPS SETUP ###########################################
from dustmaps.config import config as dust_config
import importlib

# Point dustmaps to your parent directory (works for all workers if they see the same FS)
if 'DP_path_to_dustmaps' in globals() and DP_path_to_dustmaps:
    dust_config['data_dir'] = DP_path_to_dustmaps
    # Also set env var for good measure (some setups rely on it)
    os.environ.setdefault('DUSTMAPS_PATH', DP_path_to_dustmaps)

# Normalize model name
DP_use_dustmap = (DP_use_dustmap or 'sfd').strip().lower()

# Which models require distance? Extend as needed.
_THREED_MODELS = {'bayestar', 'marshall', 'stilism'}

# Lazy singleton for the chosen dustmap query
_DUST_QUERY = None

def _make_dust_query(name: str):
    """
    Dynamically import and construct the dust map query object by name.
    Supported examples:
      - 'sfd'       -> dustmaps.sfd.SFDQuery (2D E(B-V))
      - 'bayestar'  -> dustmaps.bayestar.BayestarQuery (3D E(B-V) requires distance)
      - 'planck'    -> dustmaps.planck.PlanckQuery (returns E(B-V) proxy)
    Extend this mapper if you use other maps.
    """
    if name == 'sfd':
        from dustmaps.sfd import SFDQuery
        return SFDQuery()
    if name.startswith('bayestar'):
        # e.g. 'bayestar' or 'bayestar2019'
        from dustmaps.bayestar import BayestarQuery
        return BayestarQuery()  # configure kwargs if you need (e.g., version)
    if name == 'planck':
        from dustmaps.planck import PlanckQuery
        return PlanckQuery()
    # Fallback: try to import dustmaps.<name>.<ClassNamedLikeName>Query
    try:
        mod = importlib.import_module(f'dustmaps.{name}')
        # Heuristic: pick first attribute ending with 'Query'
        for attr in dir(mod):
            if attr.lower().endswith('query'):
                return getattr(mod, attr)()
    except Exception:
        pass
    raise ValueError(f"Unsupported DP_use_dustmap='{name}'. Known examples: 'sfd', 'bayestar', 'planck'.")

def get_dust_query():
    """Return a process-wide singleton of the chosen dust query (safe for Dask workers)."""
    global _DUST_QUERY
    if _DUST_QUERY is None:
        _DUST_QUERY = _make_dust_query(DP_use_dustmap)
    return _DUST_QUERY

Other setups.

In [ ]:
MAG_CONV = np.log(10) * 0.4

if DP_col_value_to_replace is None:
    DP_col_value_to_replace = np.nan
if DP_err_value_to_replace is None:
    DP_err_value_to_replace = np.nan


# --------------------------------------------------------------------
# Decide suffix and operational flags based on input_col_type and flags
# (keeps the same validation/error logic and operational flags)
# --------------------------------------------------------------------
ALLOWED_INPUT_TYPES = {"flux", "flux_dered", "mag", "mag_dered"}

def decide_suffix_and_flags(input_col_type: str,
                            compute_mag: bool,
                            compute_dered: bool):
    """
    Returns (suffix, WILL_COMPUTE_MAG, WILL_DEREDDEN_ON_FLUX, WILL_DEREDDEN_ON_MAG)
    and validates the chosen combination according to the defined rules.
    NOTE: 'suffix' here is just a placeholder (legacy); the final richer suffix
    will be rebuilt below.
    """

    if input_col_type not in ALLOWED_INPUT_TYPES:
        raise ValueError(f"Invalid input_col_type='{input_col_type}'. "
                         f"Allowed: {sorted(ALLOWED_INPUT_TYPES)}")

    compute_mag = bool(compute_mag)
    compute_dered = bool(compute_dered)

    # Default flags
    WILL_COMPUTE_MAG = False
    WILL_DEREDDEN_ON_FLUX = False
    WILL_DEREDDEN_ON_MAG = False

    if input_col_type == "flux":
        if compute_mag and compute_dered:
            suffix = "_mag_dered"
            WILL_COMPUTE_MAG = True
            WILL_DEREDDEN_ON_FLUX = True
        elif compute_mag and not compute_dered:
            suffix = "_mag"
            WILL_COMPUTE_MAG = True
        elif not compute_mag and compute_dered:
            suffix = "_flux_dered"
            WILL_DEREDDEN_ON_FLUX = True
        else:
            suffix = "_flux"

    elif input_col_type == "flux_dered":
        if compute_dered:
            raise ValueError("Cannot deredden an already dereddened flux (input_col_type='flux_dered').")
        if compute_mag:
            suffix = "_mag_dered"
            WILL_COMPUTE_MAG = True
        else:
            suffix = "_flux_dered"

    elif input_col_type == "mag":
        if compute_mag:
            raise ValueError("Cannot compute magnitude when input is already magnitude (input_col_type='mag').")
        if compute_dered:
            suffix = "_mag_dered"
            WILL_DEREDDEN_ON_MAG = True
        else:
            suffix = "_mag"

    elif input_col_type == "mag_dered":
        if compute_mag:
            raise ValueError("Cannot compute magnitude when input is already magnitude (input_col_type='mag_dered').")
        if compute_dered:
            raise ValueError("Cannot deredden an already dereddened magnitude (input_col_type='mag_dered').")
        suffix = "_mag_dered"

    return suffix, WILL_COMPUTE_MAG, WILL_DEREDDEN_ON_FLUX, WILL_DEREDDEN_ON_MAG


# Call the decision logic (keeps validations and operational flags)
suffix, WILL_COMPUTE_MAG, WILL_DEREDDEN_ON_FLUX, WILL_DEREDDEN_ON_MAG = decide_suffix_and_flags(
    input_col_type=input_col_type,
    compute_mag=DP_compute_magnitude,
    compute_dered=DP_compute_dereddening,
)

# --------------------------------------------------------------------
# Rebuild a richer 'suffix' for reuse:
# suffix = _{DP_which_release}_{out_kind}_{input_col_model?}_{dust_tag?}
# --------------------------------------------------------------------
# out_kind = 'mag' if the final output is in magnitudes (or input is already mag),
# otherwise 'flux'
_out_kind = 'mag' if (WILL_COMPUTE_MAG or str(input_col_type).startswith('mag')) else 'flux'

# dust_tag:
# - If dereddening is applied now → use DP_use_dustmap (lowercased)
# - If input is already *_dered and no new dereddening → 'dered'
# - Otherwise → omit
if (WILL_DEREDDEN_ON_FLUX or WILL_DEREDDEN_ON_MAG):
    _dust_tag = (str(DP_use_dustmap).strip().lower()
                 if 'DP_use_dustmap' in globals() and DP_use_dustmap
                 else 'dered')
elif str(input_col_type).endswith('_dered'):
    _dust_tag = 'dered'
else:
    _dust_tag = None

# Optional model token (if not null/empty in YAML)
if isinstance(cfg.get("input_col_model"), str):
    _model_token = cfg["input_col_model"].strip() or None
else:
    _model_token = cfg.get("input_col_model") or None

# Build the richer suffix tokens
_suffix_tokens = [str(DP_which_release), _out_kind]
if _model_token:
    _suffix_tokens.append(str(_model_token))
if _dust_tag:
    _suffix_tokens.append(_dust_tag)

# Replace old 'suffix' with this new one, prefixed by underscore
suffix = "_".join(_suffix_tokens)

# --------------------------------------------------------------------
# Define execution folder name using the new suffix
# --------------------------------------------------------------------
current_date = datetime.now().strftime('%Y-%m-%d_%H-%M')
run_folder = f"DP_run_{suffix}_{current_date}"

# --------------------------------------------------------------------
# Output paths
# --------------------------------------------------------------------
os.makedirs(user_base_path, exist_ok=True)

run_path = os.path.join(user_base_path, run_folder)
os.makedirs(run_path, exist_ok=True)

data_dir = os.path.join(run_path, 'data')
os.makedirs(data_dir, exist_ok=True)

logs_dir = os.path.join(run_path, 'logs')
os.makedirs(logs_dir, exist_ok=True)

dask_logs_dir = os.path.join(run_path, 'dask_logs')
os.makedirs(dask_logs_dir, exist_ok=True)

Making a copy of the .yaml in the logs_dir folder.

In [ ]:
shutil.copy("config.yaml", os.path.join(logs_dir, "config.yaml"))

# Initializing the Cluster

In [ ]:
if CLUSTER_extra_dask_configs:
    dask_config_class.set(CLUSTER_dask_config)
else:
    print("Running DASK with the standard memory configuration.")

In [ ]:
current_date = datetime.now().strftime('%Y-%m-%d_%H-%M')

if CLUSTER_save_the_dask_jobs_info:
    CLUSTER_job_extra_directives=[
        '--propagate',
        f'--account={CLUSTER_account}',
        f'--output={dask_logs_dir}/dask_job_%j_{current_date}.out',  
        f'--error={dask_logs_dir}/dask_job_%j_{current_date}.err',
    ]
else:
    CLUSTER_job_extra_directives=[
        '--propagate',
        f'--account={CLUSTER_account}',
        f'--output=/dev/null',  
        f'--error=/dev/null'
    ]

In [ ]:
# Configuring the SLURMCluster.
cluster = SLURMCluster(
    interface=CLUSTER_interface,         # Lustre interface
    queue=CLUSTER_queue,                 # Name of the queue
    cores=CLUSTER_cores,                 # Number of logical cores per node
    processes=CLUSTER_processes,         # Number of dask processes per node
    memory=CLUSTER_memory,               # Memory per node
    walltime=CLUSTER_walltime,           # Maximum execution time              
    job_extra_directives=CLUSTER_job_extra_directives,
)

# Scaling the cluster to use X nodes
cluster.scale(jobs=CLUSTER_dask_scale_number)

# Defining the dask client
client = Client(cluster)

# Wait for 90% of the workers to initialize
cluster.wait_for_workers(n_workers=(math.ceil(CLUSTER_dask_scale_number * CLUSTER_processes * 0.9)))
client.run(lambda: gc.collect())

In [ ]:
cluster_info = client.cluster
cluster_info

# Processing the Data

In [ ]:
input_files = [f for f in glob.glob(os.path.join(input_catalog_folder, input_catalog_pattern))]

In [ ]:
report_path = os.path.join(logs_dir, "dask-performance-report.html")

In [ ]:
def read_fits_to_df_no_fix(filename, columns=None):
    with fits.open(filename, memmap=True) as hdul:
        data = hdul[1].data
        df = pd.DataFrame(data)
        if input_user_selected_cols is not None:
            df = df[input_user_selected_cols]
    return df

In [ ]:
@delayed
def process_file_df(path):
    """
    Process one file (Parquet or FITS) and return a pandas.DataFrame.
    This function does not save to disk; it returns a processed DataFrame
    to be combined later into a global Dask DataFrame.
    """
    # Read depending on release type
    if DP_which_release == 'LSST_DP02':
        df = pd.read_parquet(path, columns=input_user_selected_cols)
    elif DP_which_release == 'DES_DR2':
        df = read_fits_to_df_no_fix(path, columns=input_user_selected_cols)
    else:
        raise ValueError(f"Unsupported DP_which_release='{DP_which_release}'")

    # Reset index if IDs are in the index
    if DP_is_id_in_the_index:
        df = df.reset_index()

    # Apply boolean column filter if requested
    if DP_filter_by_boolean_column:
        if DP_which_boolean_column not in df.columns:
            raise ValueError(f"Boolean column '{DP_which_boolean_column}' not found in DataFrame.")
        df = df[df[DP_which_boolean_column] == DP_which_value_to_keep]

    # Cross-invalidation guardrails
    if DP_replace_invalid_values and DP_cross_invalidate:
        if DP_how_to_replace_col_values != "all" or DP_how_to_replace_err_values != "all":
            raise ValueError("If DP_cross_invalidate=True, both DP_how_to_replace_* must be 'all'.")

    # If we will compute magnitude, ensure output pattern differs from input pattern
    if WILL_COMPUTE_MAG and DP_col_final_name_pattern == input_col_pattern:
        raise ValueError("Output column pattern must differ from input_col_pattern when computing magnitude.")

    # Compute E(B-V) only if needed (any dereddening on flux or mag)
    needs_ebv = bool(WILL_DEREDDEN_ON_FLUX or WILL_DEREDDEN_ON_MAG)
    if needs_ebv:
        dq = get_dust_query()
    
        # Build coordinates; for 3D maps we need a distance (per-row or fixed)
        if DP_use_dustmap in _THREED_MODELS:
            # Choose distance from column or fixed config
            dist_pc = None
            if 'DP_distance_col_pc' in globals() and DP_distance_col_pc:
                if DP_distance_col_pc not in df.columns:
                    raise ValueError(f"Distance column '{DP_distance_col_pc}' not found but DP_use_dustmap is 3D ('{DP_use_dustmap}').")
                dist_pc = df[DP_distance_col_pc].values.astype(float)
            elif 'DP_distance_fixed_pc' in globals() and DP_distance_fixed_pc:
                dist_pc = np.full(len(df), float(DP_distance_fixed_pc), dtype=float)
    
            if dist_pc is None:
                raise ValueError(
                    f"DP_use_dustmap='{DP_use_dustmap}' requires a distance. "
                    f"Provide DP_distance_col_pc (column in parsecs) OR DP_distance_fixed_pc (float parsecs)."
                )
    
            coords = SkyCoord(
                ra=df[ra_col].values * u.deg,
                dec=df[dec_col].values * u.deg,
                distance=dist_pc * u.pc
            )
        else:
            # 2D maps (e.g., SFD) need only (ra, dec)
            coords = SkyCoord(ra=df[ra_col].values * u.deg,
                              dec=df[dec_col].values * u.deg)
    
        # Query returns E(B-V) along each sightline (or proxy, depending on model)
        df["E_BV"] = dq(coords)

    # Helpers for invalid-value handling
    def get_invalid_masks(values, errors):
        invalid_val = np.zeros(len(values), dtype=bool)
        invalid_err = np.zeros(len(errors), dtype=bool)

        if DP_set_some_limit_as_invalid_for_col:
            invalid_val |= (np.abs(values) >= DP_invalid_limit_value_for_col)
        if DP_set_some_limit_as_invalid_for_err:
            invalid_err |= (np.abs(errors) >= DP_invalid_limit_value_for_err)
        if DP_is_nan_and_inf_invalid_for_col:
            invalid_val |= ~np.isfinite(values)
        if DP_is_nan_and_inf_invalid_for_err:
            invalid_err |= ~np.isfinite(errors)
        if DP_cross_invalidate and DP_how_to_replace_col_values == "all" and DP_how_to_replace_err_values == "all":
            invalid_err |= invalid_val
            invalid_val |= invalid_err

        return invalid_val, invalid_err

    def apply_replacement_locally(arr, mask, replacement_value):
        return np.where(mask, replacement_value, arr)

    flux_cols_to_drop = []

    # Loop over all bands
    for band in selected_bands:
        col_in = input_col_pattern.replace("BAND", band)
        err_in = input_err_pattern.replace("BAND", band)

        band_fmt = {
            "lower_case": band.lower(),
            "upper_case": band.upper()
        }.get(DP_pesonalized_which_band_case, band)

        final_col = DP_col_final_name_pattern.replace("BAND", band_fmt)
        final_err_col = DP_err_final_name_pattern.replace("BAND", band_fmt)

        if col_in not in df.columns or err_in not in df.columns:
            raise ValueError(f"Missing column(s) {[col_in, err_in]} in file {path}")

        # Start from the input columns
        values = df[col_in].astype(float, copy=False)
        errors = df[err_in].astype(float, copy=False)

        # 1) Deredden on flux (if requested). Applies only when working in flux space.
        if WILL_DEREDDEN_ON_FLUX:
            # A_lambda = R_band * E(B-V)
            A_lambda = df["E_BV"] * A_EBV[band]  # mag
            # Flux correction factor = 10^(+0.4 * A_lambda)
            factor = np.power(10.0, 0.4 * A_lambda)
            values = values * factor
            errors = errors * factor

        # 2) Compute magnitude (if requested). Use CURRENT flux (after any dereddening).
        if WILL_COMPUTE_MAG:
            # Keep a safe copy of current flux for error propagation
            f_curr = values
            # m = -2.5 log10(f) + MAG_OFFSET
            values = -2.5 * np.log10(f_curr) + MAG_OFFSET
            # σ_m = σ_f / (f * ln(10) * 0.4) = σ_f / (f * MAG_CONV)
            errors = errors / (f_curr * MAG_CONV)

        # 3) Deredden on magnitude (if requested). Applies only when in magnitude space.
        if WILL_DEREDDEN_ON_MAG:
            A_lambda = df["E_BV"] * A_EBV[band]
            values = values - A_lambda
            # Magnitude errors are unaffected by an additive shift

        # Mark input flux columns for dropping if we either computed mag
        # or applied dereddening in flux space and the user doesn't want to keep them.
        if (WILL_COMPUTE_MAG or WILL_DEREDDEN_ON_FLUX) and not DP_keep_flux_columns_when_computing_mag_or_dered:
            flux_cols_to_drop.extend([col_in, err_in])

        # Invalid value replacement (applies to whichever space we are in now)
        if DP_replace_invalid_values:
            invalid_val, invalid_err = get_invalid_masks(values, errors)

            if DP_how_to_replace_col_values == "all":
                values = apply_replacement_locally(values, invalid_val, DP_col_value_to_replace)
            elif DP_how_to_replace_col_values == "only_with_invalid_err":
                values = apply_replacement_locally(values, invalid_err, DP_col_value_to_replace)

            if DP_how_to_replace_err_values == "all":
                errors = apply_replacement_locally(errors, invalid_err, DP_err_value_to_replace)
            elif DP_how_to_replace_err_values == "only_with_invalid_col":
                errors = apply_replacement_locally(errors, invalid_val, DP_err_value_to_replace)

        # Optional rounding
        if DP_round_col:
            values = np.round(values, DP_round_col_decimal_cases)
        if DP_round_err:
            errors = np.round(errors, DP_round_err_decimal_cases)

        # Write final columns
        df[final_col] = values
        df[final_err_col] = errors

    # Clean up helper column if we created it
    if needs_ebv and "E_BV" in df.columns:
        df.drop(columns=["E_BV"], inplace=True)

    # Drop original flux columns if requested and applicable
    if (WILL_COMPUTE_MAG or WILL_DEREDDEN_ON_FLUX) and not DP_keep_flux_columns_when_computing_mag_or_dered:
        df.drop(columns=[c for c in set(flux_cols_to_drop) if c in df.columns], inplace=True)

    # Optionally drop the boolean filter column after filtering
    if DP_filter_by_boolean_column and DP_drop_column_after_filter:
        if DP_which_boolean_column in df.columns:
            df.drop(columns=[DP_which_boolean_column], inplace=True)

    return df

In [ ]:
%%time

# ── Build the list of delayed DataFrames (order-preserving) ──
delayed_dfs = [process_file_df(p) for p in input_files]

# Combine into a single Dask DataFrame (lazy, preserves order of delayed_dfs)
ddf = dd.from_delayed(delayed_dfs)

# Normalize config flags
has_target = ('DP_target_rows_per_part' in globals()
              and DP_target_rows_per_part is not None
              and int(DP_target_rows_per_part) > 0)

has_order = ('DP_order_by' in globals()
             and DP_order_by not in (None, False, '', []))

# Normalize DP_order_by to a list of columns (if provided)
order_cols = None
if has_order:
    if isinstance(DP_order_by, str):
        order_cols = [DP_order_by]
    elif isinstance(DP_order_by, (list, tuple)):
        order_cols = list(DP_order_by)
    else:
        raise ValueError("DP_order_by must be None, str, or list/tuple of str.")
    # Sanity check: all order columns must exist
    missing = [c for c in order_cols if c not in ddf.columns]
    if missing:
        raise ValueError(f"DP_order_by columns not found in dataframe: {missing}")

with performance_report(filename=report_path):

    # CASES:
    # 1) has_target and has_order  -> global ordering + uniform rechunk
    # 2) has_target and not has_order -> uniform rechunk only
    # 3) not has_target and has_order -> per-partition ordering only (no global shuffle)
    # 4) neither -> no ordering, no rechunk

    if has_target and has_order:
        # ---- Global ordering (ascending) ----
        # Dask supports one-column sort natively. For multi-column, chain stable sorts (reverse priority).
        if len(order_cols) == 1:
            ddf_sorted = ddf.sort_values(order_cols[0], shuffle="p2p")
        else:
            ddf_sorted = ddf
            for col in reversed(order_cols):
                ddf_sorted = ddf_sorted.sort_values(col, shuffle="p2p")

        # ---- Uniform rechunk without shuffle (order-preserving) ----
        ddf_in = ddf_sorted

        # 1) lengths and totals
        part_lengths = ddf_in.map_partitions(len).compute()
        total_rows = int(sum(int(x) for x in part_lengths))

        target_rows = int(DP_target_rows_per_part)
        n_parts = max(1, math.ceil(total_rows / target_rows))

        base = total_rows // n_parts
        remainder = total_rows % n_parts
        chunk_sizes = [(base + 1) if i < remainder else base for i in range(n_parts)]

        delayed_src_parts = ddf_in.to_delayed()
        meta = ddf_in._meta

        @delayed
        def _slice_pdf(pdf, start, end):
            """Return pdf.iloc[start:end]."""
            return pdf.iloc[start:end]

        def _concat_pieces(pieces):
            """Concat a list of delayed pandas DataFrames, preserving order."""
            return delayed(pd.concat)(pieces, ignore_index=True)

        delayed_new_parts = []
        i_part, i_off = 0, 0
        src_n = len(delayed_src_parts)

        for need in chunk_sizes:
            remaining = need
            pieces = []
            while remaining > 0 and i_part < src_n:
                part_len = int(part_lengths[i_part])
                if i_off >= part_len:
                    i_part += 1
                    i_off = 0
                    continue
                can_take = part_len - i_off
                take_now = min(remaining, can_take)
                d_piece = _slice_pdf(delayed_src_parts[i_part], i_off, i_off + take_now)
                pieces.append(d_piece)
                i_off += take_now
                remaining -= take_now
                if i_off >= part_len:
                    i_part += 1
                    i_off = 0
            if pieces:
                delayed_new_parts.append(_concat_pieces(pieces))

        ddf_out = dd.from_delayed(delayed_new_parts, meta=meta)

    elif has_target and not has_order:
        # ---- No ordering; uniform rechunk only ----
        ddf_in = ddf

        part_lengths = ddf_in.map_partitions(len).compute()
        total_rows = int(sum(int(x) for x in part_lengths))

        target_rows = int(DP_target_rows_per_part)
        n_parts = max(1, math.ceil(total_rows / target_rows))

        base = total_rows // n_parts
        remainder = total_rows % n_parts
        chunk_sizes = [(base + 1) if i < remainder else base for i in range(n_parts)]

        delayed_src_parts = ddf_in.to_delayed()
        meta = ddf_in._meta

        @delayed
        def _slice_pdf(pdf, start, end):
            return pdf.iloc[start:end]

        def _concat_pieces(pieces):
            return delayed(pd.concat)(pieces, ignore_index=True)

        delayed_new_parts = []
        i_part, i_off = 0, 0
        src_n = len(delayed_src_parts)

        for need in chunk_sizes:
            remaining = need
            pieces = []
            while remaining > 0 and i_part < src_n:
                part_len = int(part_lengths[i_part])
                if i_off >= part_len:
                    i_part += 1
                    i_off = 0
                    continue
                can_take = part_len - i_off
                take_now = min(remaining, can_take)
                d_piece = _slice_pdf(delayed_src_parts[i_part], i_off, i_off + take_now)
                pieces.append(d_piece)
                i_off += take_now
                remaining -= take_now
                if i_off >= part_len:
                    i_part += 1
                    i_off = 0
            if pieces:
                delayed_new_parts.append(_concat_pieces(pieces))

        ddf_out = dd.from_delayed(delayed_new_parts, meta=meta)

    elif (not has_target) and has_order:
        # ---- Per-partition ordering only (no global shuffle) ----
        # Uses pandas' sort_values inside each partition; preserves the original partitioning.
        def _sort_partition(pdf, cols):
            return pdf.sort_values(cols, ascending=True)

        ddf_out = ddf.map_partitions(_sort_partition, order_cols, meta=ddf._meta)

    else:
        # ---- No ordering, no rechunk ----
        ddf_out = ddf

    # ── Saving (Parquet/CSV/HDF5 via tables_io per partition) ──
    ext = {"parquet": "parquet", "csv": "csv", "hdf5": "h5"}[save_output_as]

    # Build a file-naming function consistent across formats
    def _file_name(i: int) -> str:
        return f"{suffix}_part{i}.{ext}"

    if save_output_as == "parquet":
        # Dask lets you customize the part filenames via name_function
        ddf_out.to_parquet(
            data_dir,
            write_index=False,
            name_function=_file_name  # Dask will call with i = 0..nparts-1
        )

    elif save_output_as == "csv":
        # Same idea for CSV; no need for a glob pattern when name_function is provided
        ddf_out.to_csv(
            data_dir,
            index=False,
            name_function=_file_name
        )

    elif save_output_as == "hdf5":
        # One HDF5 per partition with tables_io (parallel, contention-free)
        delayed_parts_to_write = ddf_out.to_delayed()
        tables_io_kwargs = {}  # e.g., {"complib": "blosc", "complevel": 5} if supported

        @delayed
        def _write_h5_part_with_tables_io(pdf, out_path, kwargs):
            pdf = pdf.reset_index(drop=True)
            tables_io.write(pdf, out_path, **kwargs)
            return out_path

        write_tasks = []
        for i, part in enumerate(delayed_parts_to_write):
            out_path = os.path.join(data_dir, _file_name(i))
            write_tasks.append(_write_h5_part_with_tables_io(part, out_path, tables_io_kwargs))

        written_paths = dask.compute(*write_tasks)
        print(f"✅ Wrote {len(written_paths)} HDF5 files to {data_dir} with tables_io")

print("✅ Save complete (branching logic applied).")

# Validation

## Reading the catalogs

Reading the input catalog.

In [ ]:
if DP_do_validation:
    if DP_which_release == 'LSST_DP02':
        ddf_input = dd.read_parquet(input_files, columns=input_user_selected_cols)

        if DP_filter_by_boolean_column:
            if DP_which_boolean_column not in ddf_input.columns:
                raise ValueError(f"Column '{DP_which_boolean_column}' not found in ddf_input.")
            ddf_input = ddf_input[ddf_input[DP_which_boolean_column] == DP_which_value_to_keep]

        if DP_is_id_in_the_index:
            ddf_input = ddf_input.reset_index()

    elif DP_which_release == 'DES_DR2':
        delayed_dfs = [delayed(read_fits_to_df_no_fix)(file, columns=input_user_selected_cols) for file in input_files]
        ddf_input = dd.from_delayed(delayed_dfs)

Reading the output catalog.

In [ ]:
if DP_do_validation:
    output_folder = data_dir

    if save_output_as == "parquet":
        output_paths = os.path.join(output_folder, "*.parquet")
        ddf_output = dd.read_parquet(output_paths)

    elif save_output_as == "csv":
        output_paths = os.path.join(output_folder, "*.csv")
        ddf_output = dd.read_csv(output_paths, assume_missing=True)

    elif save_output_as == "hdf5":
        output_files = glob.glob(os.path.join(output_folder, "*.h5"))
        delayed_dfs = [
            delayed(tables_io.read)(file) for file in output_files
        ]
        ddf_output = dd.from_delayed(delayed_dfs)

    else:
        raise ValueError(f"Ouput format not supported for reading: {save_output_as}")

Reading the template catalog.

In [ ]:
if DP_do_validation:
    if DP_compare_with_template:
        template_files = glob.glob(os.path.join(template_path, template_pattern))

        template_target_cols = [template_target_col.replace("BAND", band) for band in template_bands_for_comparisson]
        template_columns_to_read = [template_id_col] + template_target_cols

        if DP_template_type == "parquet":
            ddf_template = dd.read_parquet(template_files, columns=template_columns_to_read)

        elif DP_template_type == "fits":
            template_delayed_dfs = [
                delayed(read_fits_to_df_no_fix)(file, columns=template_columns_to_read)
                for file in template_files
            ]
            ddf_template = dd.from_delayed(template_delayed_dfs)

## Getting some basic informations

Printing the columns of the output dataframe and saving.

In [ ]:
if DP_do_validation:
    input_selected_columns = list(ddf_input.columns)
    print("📌 Selected columns in ddf_input:")
    print(input_selected_columns)
    
    output_columns = list(ddf_output.columns)
    print("📌 Columns in ddf_output:")
    print(output_columns)
    
    if DP_save_the_data:
        os.makedirs(logs_dir, exist_ok=True)
        columns_path = os.path.join(logs_dir, "input_output_columns.txt")
        with open(columns_path, "w") as f:
            f.write("📌 Selected columns in ddf_input:\n")
            for col in input_selected_columns:
                f.write(col + "\n")
            f.write("\n📌 Columns in ddf_output:\n")
            for col in output_columns:
                f.write(col + "\n")

Validating the sizes of input and output dataframes.

In [ ]:
if DP_do_validation:
    input_len = ddf_input.shape[0].compute()
    print(f"📥 Number of rows in input (filtered): {input_len}")

    # === Read output DataFrame ===
    output_len = ddf_output.shape[0].compute()
    print(f"📤 Number of rows in final output: {output_len}")

    # === Simple validation ===
    if input_len == output_len:
        print("✅ Row count matches!")
    else:
        print("⚠️ Row count mismatch!")
    
    match = input_len == output_len
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Validation message
    validation_text = f"""# ==== SIMPLE VALIDATION ====
    Validation timestamp: {timestamp}

    📥 Number of rows in input (filtered): {input_len}
    📤 Number of rows in final output: {output_len}

    Result: {"✅ Row count matches!" if match else "⚠️ Row count mismatch!"}
    """
    
    if DP_save_the_data:
        validation_path = os.path.join(logs_dir, "validation_info.txt")
        with open(validation_path, "w") as f:
            f.write(validation_text)

        print(f"✅ Validation result saved to: {validation_path}")

Getting some IDs in both dataframes, printing and saving.

In [ ]:
if DP_do_validation:
    selected_ids = ddf_input[id_col].head(10).tolist()

    def filter_by_ids(df, col, values):
        return df[df[col].isin(values)]

    filtered_input = ddf_input.map_partitions(filter_by_ids, id_col, selected_ids, meta=ddf_input._meta)
    filtered_output = ddf_output.map_partitions(filter_by_ids, id_col, selected_ids, meta=ddf_output._meta)

    input_sample = filtered_input.compute().sort_values(id_col)
    output_sample = filtered_output.compute().sort_values(id_col)

    print("📥 Subset of ddf_input with selected IDs:")
    display(input_sample)

    print("📤 Subset of ddf_output with the same IDs:")
    display(output_sample)

    if DP_save_the_data:
        os.makedirs(logs_dir, exist_ok=True)

        input_path = os.path.join(logs_dir, "ddf_input_sample.csv")
        output_path = os.path.join(logs_dir, "ddf_output_sample.csv")

        input_sample.to_csv(input_path, index=False)
        output_sample.to_csv(output_path, index=False)

        print(f"✅ CSV files saved to:\n- {input_path}\n- {output_path}")

Getting some IDs in both dataframes containing invalid values, printing and saving.

In [ ]:
if DP_do_validation and DP_replace_invalid_values:
    print("\n🔍 Searching for 10 objects with invalid values in the output...")

    # Get the actual output column names (col/err)
    col_pairs = []
    for band in selected_bands:
        if DP_pesonalized_which_band_case == 'lower_case':
            band_formatted = band.lower()
        elif DP_pesonalized_which_band_case == 'upper_case':
            band_formatted = band.upper()
        else:
            band_formatted = band

        col_out = DP_col_final_name_pattern.replace("BAND", band_formatted)
        err_out = DP_err_final_name_pattern.replace("BAND", band_formatted)
        col_pairs.append((col_out, err_out))

    # Identify configured invalid replacement values
    col_invalid_value = DP_col_value_to_replace
    err_invalid_value = DP_err_value_to_replace

    # Function to detect invalid rows in each partition
    def find_invalid_rows(df):
        mask = pd.Series(False, index=df.index)
        for col, err in col_pairs:
            if col in df.columns and err in df.columns:
                if pd.isna(col_invalid_value):
                    mask |= df[col].isna()
                else:
                    mask |= (df[col] == col_invalid_value)

                if pd.isna(err_invalid_value):
                    mask |= df[err].isna()
                else:
                    mask |= (df[err] == err_invalid_value)
        return df[mask].head(10)

    # Apply with Dask and collect results
    invalid_rows_dd = ddf_output.map_partitions(find_invalid_rows, meta=ddf_output._meta)
    invalid_rows = invalid_rows_dd.compute().drop_duplicates(subset=[id_col]).sort_values(id_col).head(10)

    if len(invalid_rows) == 0:
        print("✅ No objects with invalid values found in the output.")
    else:
        print(f"⚠️ Found {len(invalid_rows)} objects with invalid values in the output:")
        display(invalid_rows)

        # Fetch corresponding input objects
        ids_with_invalid = invalid_rows[id_col].tolist()
        filtered_input_invalid = ddf_input.map_partitions(filter_by_ids, id_col, ids_with_invalid, meta=ddf_input._meta)
        input_invalid_sample = filtered_input_invalid.compute().sort_values(id_col)

        print("📥 Matching entries in the input:")
        display(input_invalid_sample)

        if DP_save_the_data:
            os.makedirs(logs_dir, exist_ok=True)

            invalid_input_path = os.path.join(logs_dir, "invalid_input_sample.csv")
            invalid_output_path = os.path.join(logs_dir, "invalid_output_sample.csv")

            input_invalid_sample.to_csv(invalid_input_path, index=False)
            invalid_rows.to_csv(output_path, index=False)

            print(f"✅ Invalid value samples saved to:\n- {invalid_input_path}\n- {invalid_output_path}")

## Comparing with template catalog

In [ ]:
if DP_do_validation and DP_compare_with_template:
    if len(selected_bands_for_comparisson) != len(template_bands_for_comparisson):
        raise ValueError("selected_bands_for_comparisson and template_bands_for_comparisson must have the same length.")

    invalid_value = DP_col_value_to_replace if DP_replace_invalid_values else None

    ddf_merged = ddf_output.merge(
        ddf_template,
        left_on=id_col,
        right_on=template_id_col,
        suffixes=("_output", "_template")
    )

    comparison_lines = []
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    comparison_lines.append("# ==== GLOBAL COLUMN COMPARISON ====")
    comparison_lines.append(f"Analysis timestamp: {timestamp}")
    comparison_lines.append(f"Total band pairs compared: {len(selected_bands_for_comparisson)}\n")
    comparison_lines.append("\U0001F50D Comparing columns (using isclose with Dask):\n")

    for band_out, band_template in zip(selected_bands_for_comparisson, template_bands_for_comparisson):
        # Coluna no template
        col_template = template_target_col.replace("BAND", band_template)

        # Coluna no output
        if DP_pesonalized_which_band_case == 'lower_case':
            band_fmt = band_out.lower()
        elif DP_pesonalized_which_band_case == 'upper_case':
            band_fmt = band_out.upper()
        else:
            band_fmt = band_out
        col_out = DP_col_final_name_pattern.replace("BAND", band_fmt)

        if col_out == col_template:
            col_out = f"{col_out}_output"
            col_template = f"{col_template}_template"

        if col_out in ddf_merged.columns and col_template in ddf_merged.columns:
            if invalid_value is not None:
                valid_mask = (
                    (ddf_merged[col_out] != invalid_value) &
                    (ddf_merged[col_template] != invalid_value) &
                    ddf_merged[col_out].map_partitions(np.isfinite) &
                    ddf_merged[col_template].map_partitions(np.isfinite)
                )
            else:
                valid_mask = (
                    ddf_merged[col_out].map_partitions(np.isfinite) &
                    ddf_merged[col_template].map_partitions(np.isfinite)
                )

            ddf_valid = ddf_merged[valid_mask][[id_col, col_out, col_template]]
            total = ddf_valid[id_col].count().compute()

            is_diff = ddf_valid.map_partitions(
                lambda df: pd.Series(
                    ~np.isclose(df[col_out], df[col_template], atol=comparisson_precision, rtol=0),
                    index=df.index
                ),
                meta=pd.Series(dtype=bool)
            )
            diff_count = is_diff.sum().compute()
            percent_diff = (diff_count / total) * 100 if total > 0 else 0

            msg = f"\U0001F4CF Band {band_out.upper()} vs {band_template.upper()}: {total} valid | {diff_count} different ({percent_diff:.5f}%)"
            print(msg)
            comparison_lines.append(msg)

            if diff_count > 0:
                df_sample = ddf_valid.sample(frac=0.01, random_state=42).compute()
                diff_mask = ~np.isclose(df_sample[col_out], df_sample[col_template], atol=comparisson_precision, rtol=0)
                df_diffs = df_sample[diff_mask]

                if not df_diffs.empty:
                    preview_msg = "\n\U0001F50E Sample differences:"
                    print(preview_msg)
                    print(df_diffs.head())

                    comparison_lines.append(preview_msg)
                    comparison_lines.append(df_diffs.head().to_string(index=False))

                    if DP_save_the_data:
                        diff_path = os.path.join(logs_dir, f"diff_sample_{band_out}_vs_{band_template}.csv")
                        df_diffs.to_csv(diff_path, index=False)
        else:
            msg = f"\u26A0\uFE0F Band {band_out.upper()} vs {band_template.upper()}: missing columns -> {col_out} or {col_template}"
            print(msg)
            comparison_lines.append(msg)

    if DP_save_the_data:
        os.makedirs(logs_dir, exist_ok=True)
        comparison_path = os.path.join(logs_dir, "global_comparison.txt")
        with open(comparison_path, "w") as f:
            f.write("\n".join(comparison_lines))

        print(f"\u2705 Global column comparison saved to: {comparison_path}")

# Closing the cluster

In [ ]:
cluster.close()
client.close()